<img align="right" src="../../doc/banner_siegel.png" style="width:1000px;">

# Advanced Xarray

## Background

The Python library `xarray` simplifies working with labelled multi-dimension arrays. The library introduces labels in the forms of dimensions, coordinates and attributes on top of `numpy` arrays. This structure allows easier and more effective handling of remote sensing raster data in a Python environment. Therefore, it is essential to fully understand the structure of an `xarray`. A first introduction into the usage of `xarray` within the eo2cube environment was given in ["04_loading_data_and_basic_xarray"](04_loading_data_and_basic_xarray.ipynb). This notebook builds on this gained knowledge and attempts to give a deeper understanding of the `xarray` data structure of raster data. If you are interested in learning more about the structures of the original `xarray`, have a look at this [**"introduction to xarray" notebook**](intro_to_xarray.ipynb) within the "intro_to_python" directory.
To get more information about the `xarray` package, visit the [offical documentation website](http://xarray.pydata.org/en/stable/).

## Description

This notebook introduces users to the `xarray` library within the datacube environment. It aims to deepen the understanding of the `xarray` structure as a container for remote sensing raster data. Also it introduces useful `xarray` functions to effectivly work with raster data in the eo2cube environment. Within this notebook the following topics are covered:

* Application of built-in `xarray` functions for analyzing raster data

***

## Setting up
### Load packages

The `datacube` package is required to query the eo2cube datacube database and load the requested data. The `with_ui_cbk` function from `odc.ui` enables a progress bar when loading large amounts of data. The `xarray` and `numpy` package are needed for the different methods and analysis steps within this notebook. 

In [1]:
import datacube
from odc.ui import with_ui_cbk
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

dc = datacube.Datacube(app = "nb_understand_ndArrays")

### Datacube connection and load data

First we connect to the datacube and load an example dataset from the eo2cube.

In [3]:
# Load Data Product
ds = dc.load(product= "s2_l2a",
             x = (-2.44, -2.42),
             y = (13.58, 13.6),
             output_crs = "EPSG:32734",
             time = ("2020-06-01", "2020-06-30"),
             measurements= ["blue", "green", "red","nir"],
             resolution = (-10,10),
             group_by = "solar_day",
             progress_cbk=with_ui_cbk())

ds

<xarray.Dataset>
Dimensions:      (time: 6, y: 263, x: 259)
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-28T10:...
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 1680 1686 1686 1722 ... 1204 1204 1204 1200
    green        (time, y, x) uint16 2012 2012 2012 2002 ... 1624 1624 1612 1602
    red          (time, y, x) uint16 2442 2396 2396 2404 ... 2272 2272 2110 2176
    nir          (time, y, x) uint16 3112 3132 3132 3038 ... 2840 2840 2840 3002
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

<a id='index_array3'></a>
## **Advanced Indexing**
### 1) Temporal Subset

In the earlier tutorial we introduced `isel()`and `sel()` for indexing data. For both methods a **slicing** operator exists. If the function `slice()` is passed onto the index function, the dataset can be sliced. 
The first example uses the slicing by position method to select the first five scenes in `data_1`. The start value is included and the stop value is excluded.

#### I. Using index number

In [6]:
slice(0,5)
ds.isel(time=slice(0,5)).time
ds.isel(time = [0,1,2,3,4]).time

<xarray.DataArray 'time' (time: 5)>
array(['2020-06-03T10:47:23.000000000', '2020-06-08T10:47:19.000000000',
       '2020-06-13T10:47:23.000000000', '2020-06-18T10:47:19.000000000',
       '2020-06-23T10:47:22.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-23T10:...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00

In [7]:
ds.isel(time=slice(0,5))

<xarray.Dataset>
Dimensions:      (time: 5, y: 263, x: 259)
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-23T10:...
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 1680 1686 1686 1722 ... 985 985 928 819
    green        (time, y, x) uint16 2012 2012 2012 2002 ... 1430 1430 1366 1308
    red          (time, y, x) uint16 2442 2396 2396 2404 ... 2100 2100 1910 1834
    nir          (time, y, x) uint16 3112 3132 3132 3038 ... 2436 2436 2484 2562
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

This example uses the slicing by label method to select the scenes between "2020-01-02" and "2020-01-12". Note, that when using the `slice()` function with the `sel()` method, both start and stop value are included.

#### II. Using `datetime64` data

In [10]:
print(ds.sel(time=slice("2020-06-01","2020-06-15"))) 

#ds.sel(time=slice("2018-01-01", "2018-01-31")) # wrong time frame

<xarray.Dataset>
Dimensions:      (time: 3, y: 263, x: 259)
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-13T10:...
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 1680 1686 1686 1722 ... 1118 1118 1036 1022
    green        (time, y, x) uint16 2012 2012 2012 2002 ... 1592 1592 1554 1488
    red          (time, y, x) uint16 2442 2396 2396 2404 ... 2246 2246 2110 2054
    nir          (time, y, x) uint16 3112 3132 3132 3038 ... 2718 2718 2686 2810
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref


In [13]:
ds.sel(time=slice("2020-05-31", "2020-06-02")).time

<xarray.DataArray 'time' (time: 0)>
array([], dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00

#### III. Using other time dimensions

`xarray` also includes some useful features for the inspection of the time dimension. It allows to easily extract additional information from a dataset. The following code automatically groups the time dimension in seasons ("DJF", "MAM", JJA", "SON"). Since `ds` only contains scens from winter months, only the label "DJF" will appear. There are a lot of other `time` dimensions arguments, e.g. `month`, `week`, `weekday`, `dayofyear`.

In [25]:
#ds.time
#using special .dt accessor
#ds.time.dt

In [14]:
ds.time.dt.season

<xarray.DataArray 'season' (time: 6)>
array(['JJA', 'JJA', 'JJA', 'JJA', 'JJA', 'JJA'], dtype='<U3')
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-28T10:...
    spatial_ref  int32 32734

It is also possible to extract the "day of year" for a time step.

In [15]:
ds.time.dt.dayofyear

<xarray.DataArray 'dayofyear' (time: 6)>
array([155, 160, 165, 170, 175, 180])
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-28T10:...
    spatial_ref  int32 32734

In [16]:
ds.groupby('time.season').mean()

<xarray.Dataset>
Dimensions:      (y: 263, x: 259, season: 1)
Coordinates:
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734
  * season       (season) object 'JJA'
Data variables:
    blue         (season, y, x) float64 2.543e+03 2.524e+03 ... 2.581e+03
    green        (season, y, x) float64 2.912e+03 2.908e+03 ... 2.98e+03
    red          (season, y, x) float64 3.282e+03 3.223e+03 ... 3.411e+03
    nir          (season, y, x) float64 4.108e+03 4.118e+03 ... 4.079e+03

### 2) Spatial Subset
It is possible to index and **slice within the x and y dimensions**. The following example selects the value for each band of the pixel in the second colum of the raster and the fifth row of the raster (`x=2,y=5`)

In [17]:
ds.isel(x=2, y=5)
#ds.isel(x=[0,1,2], y=5)

<xarray.Dataset>
Dimensions:      (time: 6)
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-28T10:...
    y            float64 1.163e+07
    x            float64 -2.101e+06
    spatial_ref  int32 32734
Data variables:
    blue         (time) uint16 1670 928 867 9864 735 857
    green        (time) uint16 1980 1282 1360 10088 1104 1290
    red          (time) uint16 2422 1594 1668 10176 1290 1462
    nir          (time) uint16 3142 2642 3090 10584 3006 3252
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

### 3) Combining Temporal and Spatial Subset

Again, this method can be combined with the `slice()` operator to do a spatial subset of the dataset based on the position of the pixels. If you know the actual coordinate (x,y) value (extent) of the spatial subset area, use the `sel()` function.
Additionally, this subset can also be sliced in the time dimensions.

The following example subsets the `ds` by the spatial location of the pixels. Only the pixels from the first to the fifth column and the pixels from the first to the fifth row are included in the output. Also the scenes where filtered in the time dimension between the first and fifth time step.

In [18]:
ds2 = ds.isel(time=slice(0,5), x= slice(0,5), y=slice(0,5))
ds2

#ds2.time
#plt.scatter(ds2.x.values, ds2.y.values)

<xarray.Dataset>
Dimensions:      (time: 5, y: 5, x: 5)
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-23T10:...
  * y            (y) float64 1.163e+07 1.163e+07 1.163e+07 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.101e+06 -2.101e+06
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 1680 1686 1686 1722 ... 641 697 679 804
    green        (time, y, x) uint16 2012 2012 2012 2002 ... 1024 1036 1072 1170
    red          (time, y, x) uint16 2442 2396 2396 2404 ... 998 1086 1176 1454
    nir          (time, y, x) uint16 3112 3132 3132 3038 ... 3288 3122 3054 2758
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

## **Data Manipulation & Statistics**

This notebook presents some basic built-in functions of the `xarray` library to manipulate and transform data in a `xarray.Dataset`. In this notebook only a fraction of the available `xarray` functions are presented. For a complete overview of all the available functions and tools of the `xarray` package please visit the [documentation website](http://xarray.pydata.org/en/stable/). The [notebook 07](07_basic_analysis.ipynb) will cover this topic with a focus on an application oriented remote sensing approach.

###  1) Statistical Operation

The simple built-in functions allow the user to do simple calculations with a `xarray.Dataset`.
The **basic math** built-in `xarray` functions are:
* `min()`, `max()`
* `mean()`, `median()`
* `sum()`
* `std()`

The following code demonstrates the easy use of the `max()` function to extract the maximum value of the red band in the `ds` dataset.

In [19]:
print(ds.red.max())

<xarray.DataArray 'red' ()>
array(10392, dtype=uint16)
Coordinates:
    spatial_ref  int32 32734


To apply a function to every value of a specified dimension (e.g. to calculate the mean of every time step) the `dim` argument in the basic math function must be define with the dimension label.

This examples calculates the mean of the `red` band for each pixel (defined by the unique `x`, `y` combination) over every time step. The result is a data array which can be used for further time series visualization and analysis.

In [20]:
print(ds.red.mean(dim=["x", "y"]))

#ds.red.mean(dim=["x", "y"]).values
#plt.plot(ds.red.mean(dim=["x", "y"]).values)

<xarray.DataArray 'red' (time: 6)>
array([ 2580.33642116,  2004.10056227,  2255.1672857 , 10208.46801826,
        1852.59597457,  2229.75583188])
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-28T10:...
    spatial_ref  int32 32734


This examples works the other way around. It calculates the standard deviation of every pixel (`x`, `y`) over all timesteps of the dataset `ds`.

In [21]:
print(ds.red.std(dim="time"))

<xarray.DataArray 'red' (y: 263, x: 259)>
array([[3103.1911353 , 3140.89132785, 3140.89132785, ..., 3006.90065608,
        3043.74827401, 3062.33375059],
       [3160.7499989 , 3192.69979798, 3140.38681269, ..., 3012.14130184,
        3043.74827401, 3084.02954302],
       [3158.69607416, 3210.20366263, 3171.66069364, ..., 2996.18878281,
        2996.18878281, 3014.36223216],
       ...,
       [3087.08033585, 3119.66593653, 3166.06646944, ..., 2897.90416604,
        2955.75132393, 2972.42654334],
       [3087.08033585, 3128.83229905, 3128.83229905, ..., 2964.65145038,
        2981.38237885, 3003.81535163],
       [3078.31283155, 3114.47404156, 3142.45997058, ..., 2926.41223268,
        2961.02542298, 2996.72059647]])
Coordinates:
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734


Remember, to access the raw `numpy` array that stores the values of the resulting `xarray.DataArrays`, the suffix `.values` is needed. This allows you to work with the "actual" data values.

In [22]:
print(ds.blue.sum(dim=["x","y"]).values)
#plt.plot(ds.blue.sum(dim=["x","y"]).values)

[128485774  74802587  78872126 672045504  66974152  86695536]


### 2) Conditional Operation

Using conditional operation can be very helpful when we need to analyse satellite scenes or pixels lies within our interests. The `where()` function provides the option to **mask** a `xarray.Dataset` based on a logical condition. By default, the function converts all values that match the condition to NaN values. This is extremly useful when applied in combination with a binary mask to mask your data to the desired values. The argument `other` let´s you define a subset value for all values that match the condition (default is `nan`). The argument `drop` drops all values which do not match with the condition.
The following example masks the datatset `ds` to only the values which have a reflectance value of greater than 700 in the `red` band.

In [23]:
print(ds.where(ds.red > 700))
#print(ds.where(ds.red < 700))

<xarray.Dataset>
Dimensions:      (time: 6, y: 263, x: 259)
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-28T10:...
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) float64 1.68e+03 1.686e+03 ... 1.204e+03 1.2e+03
    green        (time, y, x) float64 2.012e+03 2.012e+03 ... 1.602e+03
    red          (time, y, x) float64 2.442e+03 2.396e+03 ... 2.11e+03 2.176e+03
    nir          (time, y, x) float64 3.112e+03 3.132e+03 ... 2.84e+03 3.002e+03
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref


This code subsets all zeros in the red band of the dataset `ds` in the first time stamp with the new value -9999.

In [24]:
replace = ds.red.isel(time=0).where(ds.red != 0, other = -9999)
#replace.values.min()

The implemented `xarray` function `isin()` allows to **test each value** of `xarray.Dataset` or `xarray.DataArray` whether it is in the elements defined within the function. It returns a boolean array which can be used as a mask.
This example checks all the values of the `red` measurement if the value is in an array from 0 to 550.

In [25]:
mask_red = ds.red.isin(range(550))
print(mask_red)

#plt.imshow(mask_red) #error
#plt.imshow(mask_red.isel(time=3))

<xarray.DataArray 'red' (time: 6, y: 263, x: 259)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        ...,
        [False, False, False, ...,

The created mask can easily be combined with the `where()` function to filter the dataset based on the predefined mask. In this case the `ds` dataset is masked with previously defined mask `mask_red`, which was based on a logical test if values of the `red` band are within a certain range of values.

In [26]:
print(ds.where(mask_red)) #masking

<xarray.Dataset>
Dimensions:      (time: 6, y: 263, x: 259)
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-28T10:...
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    green        (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    red          (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    nir          (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref


### 3) Resampling
Working with time series data, resampling is necessary if we want the data product aligns with the temporal window are interested in.

 - **resample()**

The **`resample()` method** allows to summarise the `xarray.Dataset` to bigger or smaller chunks based on a dimension. It handels both upsampling and downsampling. The argument `time` needs to be defined like a datetime-like coordinate. In the following example we resample the `ds` dataset to a monthly time intervall (`time = "m"`) and than calculate the median value for every resample chunk. _(this process takes a little while to run)_

In [27]:
print(ds.resample(time='m').median())

<xarray.Dataset>
Dimensions:      (time: 1, y: 263, x: 259)
Coordinates:
  * time         (time) datetime64[ns] 2020-06-30
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) float64 993.0 974.0 974.0 ... 1.128e+03 1.111e+03
    green        (time, y, x) float64 1.431e+03 1.422e+03 ... 1.556e+03
    red          (time, y, x) float64 1.892e+03 1.856e+03 ... 2.11e+03 2.115e+03
    nir          (time, y, x) float64 2.838e+03 2.856e+03 ... 2.899e+03


 - **groupby() method**

The **`groupby()` method** can also be used within the `xarray` library to *aggregate data over time*. Time aggregation arguments can be e.g. "time.year", "time.season", "time.month", "time.week", "time.day".
The code below groups the `ds` dataset into two groups by year. Therefore, a new "dimension" `year` is created. Then the median for each `year` is calculated. _(this process takes a little while to run)_

In [28]:
print(ds.groupby("time.year").median(dim="time"))

<xarray.Dataset>
Dimensions:      (y: 263, x: 259, year: 1)
Coordinates:
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734
  * year         (year) int64 2020
Data variables:
    blue         (year, y, x) float64 993.0 974.0 974.0 ... 1.128e+03 1.111e+03
    green        (year, y, x) float64 1.431e+03 1.422e+03 ... 1.556e+03
    red          (year, y, x) float64 1.892e+03 1.856e+03 ... 2.11e+03 2.115e+03
    nir          (year, y, x) float64 2.838e+03 2.856e+03 ... 2.899e+03


### 4) Interpolation
Interpolation is a common solution dealing with missing remote sensing data, either caused by coarse temporal resolution of the satellite, high cloud cover or bad quality of the scenes. For example, sometimes a scene of a specfic date is not available in the dataset. With the implemented `interp()` it is possible to **interpolate data** for predefined time steps. The function takes the next usable scene before and after the specified date and interpolates their values (default interpolation method is "linear") to build a new `xarray.Dataset`.

In this example, the `ds` dataset has missing scenes on the "2020-06-15". The `interp()` function builds a "new" scene based on an linear interpolation from the two measurments before and after the new time step.

In [29]:
print(ds.time)

<xarray.DataArray 'time' (time: 6)>
array(['2020-06-03T10:47:23.000000000', '2020-06-08T10:47:19.000000000',
       '2020-06-13T10:47:23.000000000', '2020-06-18T10:47:19.000000000',
       '2020-06-23T10:47:22.000000000', '2020-06-28T10:47:19.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-28T10:...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00


In [31]:
ds_interp = ds.interp(time=["2020-06-15"])
print(ds_interp)

<xarray.Dataset>
Dimensions:      (y: 263, x: 259, time: 1)
Coordinates:
  * y            (y) float64 1.163e+07 1.163e+07 ... 1.163e+07 1.163e+07
  * x            (x) float64 -2.101e+06 -2.101e+06 ... -2.099e+06 -2.099e+06
    spatial_ref  int32 32734
  * time         (time) datetime64[ns] 2020-06-15
Data variables:
    blue         (time, y, x) float64 3.726e+03 3.702e+03 ... 3.712e+03
    green        (time, y, x) float64 4.102e+03 4.128e+03 ... 4.115e+03
    red          (time, y, x) float64 4.474e+03 4.417e+03 ... 4.548e+03
    nir          (time, y, x) float64 5.267e+03 5.272e+03 ... 5.09e+03 5.186e+03
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref


The `merge()` function allows to **merge/join** `xarray.Datasets` or variables. By default the `merge()` function uses an "inner" join as merging operation. 
In our example the interpolated `xarray.Dataset` created above is merged to the `ds` dataset by using the default `merge()` function.

In [32]:
print(ds.merge(ds_interp).time)

<xarray.DataArray 'time' (time: 7)>
array(['2020-06-03T10:47:23.000000000', '2020-06-08T10:47:19.000000000',
       '2020-06-13T10:47:23.000000000', '2020-06-15T00:00:00.000000000',
       '2020-06-18T10:47:19.000000000', '2020-06-23T10:47:22.000000000',
       '2020-06-28T10:47:19.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-06-03T10:47:23 ... 2020-06-28T10:...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00


The `xarray` package contains a variety of other useful functions in addition to those shown here. There is a function for almost every operation needed in data analysis. For more information about the `xarray` package visit the [documentation website](http://xarray.pydata.org/en/stable/) or work through the [notebook]() in the "intro_to_python" folder.

The `xarray.Datasets` in the eo2cube datacube environment are a useful and effective structure for handeling remote sensing raster data. In this notebook you learned the basic structure and application methods of `xarray.Datasets`and `xarray.DataArrays`. However, it is very useful to not only work with the "raw" datasets and values. Sometimes it is necessary to get a visual overview of the data. The next [notebook](05_plotting_basics.ipynb) will cover how to plot `xarray` raster data nicely and efficient. This is a very useful application, as it is often more convenient to visualize the raster data.

## Recommended next steps

To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/01_jupyter_introduction.ipynb)
2. [eo2cube](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/02_eo2cube_introduction.ipynb)
3. [Loading Data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/03_data_lookup_and_loading.ipynb)
4. [Xarray I: Data Structure](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/04_xarrayI_data_structure.ipynb)
5. ***Xarray II: Index and Statistics (this notebook)***
6. [Plotting data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/06_plotting_basics.ipynb)
7. [Spatial analysis](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/07_basic_analysis.ipynb)
8. [Parallel processing with Dask](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/08_parallel_processing_with_dask.ipynb)

The additional notebooks are designed for users to build up both basic and advanced skills which are not covered by the beginner's guide. Self-motivated users can go through them according to their own needs. They act as complements for the guide:
<br>

1. [Python's file management tools](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/I_file_management.ipynb)
2. [Image Processing basics using NumPy and Matplotlib](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/II_numpy_image_processing.ipynb)
3. [Vector Processing](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/III_process_vector_data.ipynb)
4. [Advanced Plotting](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/IV_advanced_plotting.ipynb)

***

## Additional information

<font size="2">This notebook for the usage in the Open Data Cube entities of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/), is adapted from [Geoscience Australia](https://github.com/GeoscienceAustralia/dea-notebooks), published using the Apache License, Version 2.0. Thanks! </font>

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** January 2022